In [ ]:
WOS_CREDENTIALS = {
    "url": "https://zen-cpd-zen.cpd-outcomes-00-57d48117af6edd64ea27fa1142b91b0b-0000.us-south.containers.appdomain.cloud",
    "username": "admin",
    "password": "********"
}

Datamart ID of the OpenScale instance. This can be found on the information page for any existing configured model.

In [ ]:
DATAMART_ID = '00000000-0000-0000-0000-1616425950709603'

In [ ]:
!pip install ibm-cloud-sdk-core --no-cache | tail -n 1
!pip install --upgrade ibm-watson-machine-learning --user | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

In [ ]:
import os
token = os.environ['USER_ACCESS_TOKEN']

WML_CREDENTIALS = {
   "token" : token,
   "instance_id" : "wml_local",
   "url": WOS_CREDENTIALS["url"],
   "version": "3.5"
}

In [ ]:
MODEL_NAME = 'Mortgage Default - Production'
DEPLOYMENT_NAME = 'Mortgage Default - Production'

In [ ]:
import sklearn
sklearn.__version__

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

In [ ]:
space_details = wml_client.spaces.list()

In [ ]:
space_name = "Mortage Default Production"
spaces = wml_client.spaces.get_details()['resources']
space_id = None
for space in spaces:
    if space['entity']['name'] == space_name:
        space_id = space["metadata"]["id"]
        print('Using existing space')
if space_id is None:
    print('Creating new space')
    space_id = wml_client.spaces.store(
        meta_props={wml_client.spaces.ConfigurationMetaNames.NAME: space_name})["metadata"]["id"]
wml_client.set.default_space(space_id)

In [ ]:
import time
time.sleep(10)
wml_client.repository.list_models()

In [ ]:
for deployment in wml_client.deployments.get_details()['resources']:
    deployment_id=deployment['metadata']['id']
    deployment = wml_client.deployments.get_details(deployment_id)
    model_uid = deployment['entity']['asset']['id']
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deleting existing deployment with id', deployment_id)
        wml_client.deployments.delete(deployment_id)
        print('Deleting existing model with id', model_uid)
        wml_client.repository.delete(model_uid)

In [ ]:
import pandas as pd

url = 'https://raw.githubusercontent.com/emartensibm/mortgage-default/master/Mortgage_Full_Records.csv'
df_raw = pd.read_csv(url)
df = df_raw.drop('ID', axis=1)
df.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
categorical_features = ['AppliedOnline','Residence','Location']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

scaled_features = ['Income','Yrs_at_Current_Address','Yrs_with_Current_Employer',\
                   'Number_of_Cards','Creditcard_Debt','Loan_Amount','SalePrice']
scale_transformer = Pipeline(steps=[('scale', MinMaxScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('scaler', scale_transformer, scaled_features)
    ]
)

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier())])

In [ ]:
X = df.drop('MortgageDefault', axis=1)
y = df['MortgageDefault']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)

model = clf.fit(X_train, y_train)
res_predict = model.predict(X_test)
print("model score: %.3f" % clf.score(X_test, y_test))
print(classification_report(y_test, res_predict, target_names=["False", "True"]))

In [ ]:
wml_client.software_specifications.list()

In [ ]:
sw_spec_uid = wml_client.software_specifications.get_uid_by_name("default_py3.7")
sw_spec_uid

In [ ]:
metadata = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME,
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid,
    wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.23'
}

# Name the columns
cols=["Income","AppliedOnline","Residence","Yrs_at_Current_Address","Yrs_with_Current_Employer",\
      "Number_of_Cards","Creditcard_Debt","Loans","Loan_Amount","SalePrice","Location"]
      
saved_model = wml_client.repository.store_model(model=model, meta_props=metadata, training_data=X_train,\
                                                training_target=y_train, feature_names=cols,\
                                                label_column_names=["MortgageDefault"] )

saved_model

In [ ]:
model_uid = saved_model['metadata']['id']
print("Deploying model", model_uid)

meta_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_NAME,
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

deployment = wml_client.deployments.create(artifact_uid=model_uid, meta_props=meta_props)
deployment_uid = wml_client.deployments.get_uid(deployment)

In [ ]:
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))

# OpenScale

In [ ]:
from ibm_watson_openscale import APIClient
from ibm_watson_openscale.utils import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.supporting_classes.enums import *

import json
import requests
import base64
from requests.auth import HTTPBasicAuth

In [ ]:
authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS['url'],
        username=WOS_CREDENTIALS['username'],
        password=WOS_CREDENTIALS['password'],
        disable_ssl_verification=True
    )

wos_client = APIClient(service_url=WOS_CREDENTIALS['url'], service_instance_id=DATAMART_ID, authenticator=authenticator)
wos_client.version

In [ ]:
added_service_provider_result = wos_client.service_providers.add(
        name="Mortage Default Production Space",
        description="",
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        deployment_space_id = space_id,
        operational_space_id = "production",
        credentials=WML_CREDENTIALS,
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id

In [ ]:
DEPLOYMENT_UID = ''
deployments_list = wml_client.deployments.get_details()
for deployment in deployments_list["resources"]:
    if deployment['metadata']['name'] == DEPLOYMENT_NAME:
        DEPLOYMENT_UID = deployment['metadata']['id']
print(DEPLOYMENT_UID)

In [ ]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=DATAMART_ID, service_provider_id=service_provider_id, deployment_id=DEPLOYMENT_UID, deployment_space_id = space_id).result['resources'][0]
asset_deployment_details

In [ ]:
model_asset_details_from_deployment=wos_client.service_providers.get_deployment_asset(data_mart_id=DATAMART_ID,service_provider_id=service_provider_id,deployment_id=DEPLOYMENT_UID,deployment_space_id=space_id)


In [ ]:
feature_columns = ['AppliedOnline','Residence','Location','Income','Yrs_at_Current_Address','Yrs_with_Current_Employer',\
                   'Number_of_Cards','Creditcard_Debt','Loan_Amount','Loans','SalePrice']
cat_features = ['AppliedOnline','Residence','Location']

In [ ]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=DATAMART_ID,
        service_provider_id=service_provider_id,
        asset=Asset(
            asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
            name=model_asset_details_from_deployment["entity"]["asset"]["name"],
            url=model_asset_details_from_deployment["entity"]["asset"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=asset_deployment_details['metadata']['guid'],
            name=asset_deployment_details['entity']['name'],
            deployment_type= DeploymentTypes.ONLINE,
            url=asset_deployment_details['entity']['scoring_endpoint']['url']
        ),
        asset_properties=AssetPropertiesRequest(
            label_column="MortgageDefault",
            probability_fields=["probability"],
            prediction_field="prediction",
            feature_fields = feature_columns,
            categorical_fields = cat_features,
            training_data_reference=TrainingDataReference(type="cos",
                                                          location=COSTrainingDataReferenceLocation(bucket = "faststartlab-donotdelete-pr-nhfd4jnhlxgpc7",
                                                                                                    file_name = "Mortgage_Full_Records.csv"),
                                                          connection=COSTrainingDataReferenceConnection.from_dict({
                                                                        "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/7d8b3c34272c0980d973d3e40be9e9d2:2883ef10-23f1-4592-8582-2f2ef4973639::",
                                                                        "url": "https://s3.us.cloud-object-storage.appdomain.cloud",
                                                                        "api_key": "yqcPbWZ0AQPHleHVerrR4Wx5e9pymBdMgydbEra5zCif",
                                                                        "iam_url": "https://iam.ng.bluemix.net/oidc/token"}))
        )
    ).result
subscription_id = subscription_details.metadata.id
print('subscription_id: ' + subscription_id)

In [ ]:
!rm mortgage_feed.json
!wget https://raw.githubusercontent.com/emartensibm/mortgage-default/master/mortgage_feed.json

In [ ]:
import json

with open('mortgage_feed.json', 'r') as scoring_file:
    data = json.load(scoring_file)
data['fields'][1:]

In [ ]:
scoring_payload = {wml_client.deployments.ScoringMetaNames.INPUT_DATA:
                   [
                       {
                           "fields": data['fields'][1:],
                           "values": [],
                           "meta": {
                               "fields": ["ID"],
                               "values": []
                           }
                       }
                    ]
                  }

In [ ]:
import random
import string

letters = string.digits

for _ in range(0, 101):
    value_to_score = random.choice(data['values'])
    scoring_payload['input_data'][0]['values'].append(value_to_score[1:])
    scoring_payload['input_data'][0]['meta']['values'].append([int(''.join(random.choices(letters, k=8)))])
print(len(scoring_payload['input_data'][0]['values']))

In [ ]:
predictions = wml_client.deployments.score(deployment_uid, scoring_payload)
print(predictions)

In [ ]:
time.sleep(5)

payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id:", payload_data_set_id)

In [ ]:
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    raise Exception("Payload logging did not happen!")

In [ ]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)


target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)

parameters = {
    "min_samples": 100,
    "drift_threshold": 0.05,
    "train_drift_model": True,
    "enable_model_drift": True,
    "enable_data_drift": True
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
drift_monitor_instance_id

In [ ]:
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 50
}
thresholds = [
    {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
    }
]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds 
).result
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_id = explainability_details.metadata.id

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "features": [
        {
                "feature": "AppliedOnline",
                "majority": ["NO"],
                "minority": ["YES"]
        }
    ],
    "favourable_class": ["DENIED"],
    "unfavourable_class": ["APPROVED"],
    "min_records": 100
}
thresholds = [
    {
        "metric_id": "fairness_value",
        "specific_values": [
           {
                "applies_to": [
                    {
                        "type": "tag",
                        "value": "AppliedOnline",
                        "key": "feature"
                    }
                ],
                "value": 90
            }
        ],
        "type": "lower_limit",
        "value": 90
    }
]
fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

fairness_monitor_instance_id = fairness_monitor_details.metadata.id